# Fully Connected Networks

Fully connected network란, 뭐, 공식적인 이름은 아닌데요, fully connected layer로만 이루어진 네트워크를 말합니다. Activation function이 자유로운 multi-layer perceptron이라고 보시면 됩니다.

![image](./resources/fully-connected-networks.jpg)

들어가기에 앞서서, 다음 코드는 GPU가 여러개 있을 때, cuda 라이브러리가 2번 GPU만 보도록 하게끔 하는 코드인데요, cuda를 사용하는 라이브러리에게 환경변수 `CUDA_VISIBLE_DEVICES`를 통해 라이브러리가 어떤 GPU만 사용해야 하는지 알려줄 수 있습니다.

주의할 점은 tensorflow나 pytorch같은 cuda를 사용하는 라이브러리 임포트 전에 이 환경변수를 설정해 주어야 합니다.

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

우선 필요한 모듈 임포트해보겠습니다. 편의를 위해서 이전 노트북(01-Tensorflow-Basic)에서 보셨던 데이터를 로드하고 전처리(standardization)하는 코드는 ``data_utils``라는 모듈에 모아놨습니다.
이제, 그것을 단순히 호출한 하면 됩니다.

In [2]:
import tensorflow as tf
import numpy as np

from tensorflow.keras import models, layers, losses, metrics, optimizers, losses

from utils.data_utils import *

In [3]:
X_train, y_train, X_test, y_test = load_mnist_data()

이미 여러분은 이전 노트북(01-Tensorflow-Basic)에서 이미 fully connected network를 보셨습니다. 다음 코드가 바로 그 코드인데요.

In [4]:
# 먼저, 이미지를 벡터로 펼치고,
# 128개의 hidden unit을 가지는 dense(fully connected) layer
# 128개의 hidden unit을 가지는 dense(fully connected) layer
# 10개의 hidden unit을 가지는 dense(fully connected) layer

myfirstmodel = models.Sequential([
    layers.Flatten(),
    layers.Dense(128, activation=tf.nn.tanh),
    layers.Dense(128, activation=tf.nn.tanh),
    layers.Dense(10, activation=tf.nn.softmax),
])

코드에 대해서 설명을 드려보면,

- ``models.Sequential``: ``models``은 ``tensorflow.keras`` 모듈 아래에 존재하는 모듈로, 그 모듈 안에 ``Sequential``이라는 클래스가 존재합니다. ``Sequential``은 ``tensorflow.keras.models.Model``을 상속하며, sequential한 레이어를 이용하여 간단한 모델을 구축할 때 이용됩니다(이름 그대로 sequential 입니다). Sequential한 레이어를 이용한다는 것은, ``Sequential``에 데이터가 들어가면, ``Sequential``을 구성하고 있는 레이어를 차례차례, sequential하게 지나서 결과로 나오게 됩니다. 이 경우, 데이터가 들어가면, ``Flatten`` 레이어를 지나서 ``Dense`` 3개를 지나서 결과로 나오게 되는 것이죠. 이처럼 레이어를 sequential하게 쌓는 간단한 모델을 구현할 때 유용한 것이 ``Sequential`` 객체입니다.
- ``layers.Flatten()``: 데이터를 flatten 시켜주는 레이어로, 데이터가 (BATCH_SIZE, H, W, C)처럼 고차원으로 들어왔을 경우, 배치 차원을 제외한 차원을 모두 flatten시켜줍니다. 즉, (BATCH_SIZE, H\*W\*C) 차원으로 결과가 나오게 됩니다. 하나의 샘플의 차원이 (H, W, C)에서 (H\*W\*C)인 벡터가 되버리는 것이죠.
- ``layers.Dense(128, activation=tf.nn.tanh)``: fully connected layer입니다. 데이터를 입력으로 받아서, 128개의 뉴런의 아웃풋을 만들어내는 fully connected layer입니다. 이때, 마지막 결과를 내기 전에 ``activation=tf.nn.tanh``가 적용됩니다. ``activation=None``으로 두면 linear activation(activation이 없는 효과)의 효과를 만들 수 있습니다.

이번에는 `.fit`함수를 쓰지 않고 다른 방법으로 학습 루프를 만들어 볼겁니다. `.fit`함수는 편하긴 하지만, 학습을 자유롭게 커스터마이징하기에는 한계가 있다고 생각합니다. 그래서, 이번에는 `tf.GradientTape`이라는 객체를 만들어서 학습하는 법을 적어볼까 합니다.

`tf.GradientTape`객체는 일종의 scope로, `tf.GradientTape`가 열린(open)동안에는 모든 텐서 연산에 대해서 partial derivative를 계산하게 됩니다. 텐서플로는 기본적으로 그냥 텐서 연산을 하면 gradient를 계산하지 않습니다. 반드시 `tf.GradientTape`을 열고 텐서 연산을 해야 내부적으로 gradient를 계산해줍니다. 사용법은 다음과 같습니다.

```python
with tf.GradientTape() as tape:
    # some tensor operation or forward propagation
    preds = model(x_train)
    # compute loss
    loss = compute_loss(y_train, preds)
    
grads = tape.gradient(loss, model.trainable_variables)
```

`with`구문을 사용하여 `tf.GradientTape`를 열어주었는데요, 열린 상태에서 model을 호출하여 forward propagation을 수행합니다. 그래야, model의 각 weight들이 partial derivative를 계산할 수 있게 됩니다. 그리고, forward propagation의 결과를 이용하여 loss를 계산해줍니다.

여기까지 한 뒤, `tf.GradientTape`를 닫습니다. `tf.GradientTape`객체는 열린 상태에서부터 시작해서 닫힐때 까지 텐서들의 연산을 모두 기억하고 있습니다(특수한 경우도 있지만, 여기서는 넘어가겠습니다). `tf.GradientTape`는 기억하고 있는 연산을 바탕으로 gradient를 계산할 수 있습니다. 그 코드가 `tape.gradient(...)`코드입니다.

`tape.gradient`는 두 개의 인자를 받습니다. 첫번째는 미분 대상이 되는 값으로 보통 loss가 들어갈 수 있겠죠. 두 번째 인자는 첫 번째 인자를 미분할 변수들입니다. 두 번째 인자는 tensor들의 리스트가 들어가야 하며, `tape.gradient`가 계산해서 반환하는 gradients는 `tape.gradient`의 두 번째 인자로 넘어간 변수들과 똑같은 순서가 됩니다.

주의할 점은, `tape`를 사용하여 한번 gradient를 계산하고 나면, tape는 앞서 forward propagation의 과정에서 weight들로부터 얻었던 모든 정보를 폐기합니다. 따라서, `tape.gradient`를 두 번 호출하게 되면 에러가 발생합니다. 그러나, 나중에 두 번 이상 호출해야 할 때가 생기는데요, 이 문제는 그때 가서 알아보겠습니다.

이제 학습 함수를 정의해보겠습니다. `.fit`함수를 쓰지 않기 때문에 그 내부 코드를 우리가 직접 구현해야 합니다. 하지만, 직접 구현하는만큼 자유도가 높죠.

In [5]:
def train_step(x, y):
    """
    학습 함수
    
    Arguments:
    ----------
    - x: 이미지 배치 (60000, 28, 28, 1) 또는 (60000, 28, 28) 또는 (60000, 784)
    - y: 각각의 이미지의 정답 라벨
    
    Returns:
    --------
    - loss: 학습 때의 loss값
    """
    
    # loss를 0으로 초기화
    loss = 0.0
    
    # gradient를 계산하기 위해서 tape 객체를 생성하고 open합니다.
    with tf.GradientTape() as tape:
        # forward propagation
        preds = myfirstmodel(x)
        # loss함수로 loss계산
        loss += criterion(y, preds)
        
    # tape를 이용하여 loss를 trainable variable에 대해 gradient를 계산
    grads = tape.gradient(loss, myfirstmodel.trainable_variables)
    # 계산한 gradient로 trainable variable 업데이트
    optimizer.apply_gradients(zip(grads, myfirstmodel.trainable_variables))
    
    return loss.numpy()

이전에 `.fit`함수를 사용했을 때는, optimizer객체를 그냥 넘겨주기면 하면 됬었습니다. 그러나, 여기서는 gradient를 계산해주고 그것을 optimizer를 통해 직접 적용까지 시켜주어야 하는데요, 그러한 역할을 하는 메소드가 `optimizer.aaply_gradients`입니다. 이 메소드는 인자로 다음과 같은 **튜플의 리스트**를 받습니다.

```python
[
    (grad_w1, w1), (grad_w2, w2), (grad_b1, b1), ...
]
```

각 튜플은 (gradient for weights, weights)로 구성되어 있습니다. 튜플의 두 번째 원소는 trainable variable의 한 원소이고, 첫 번째 원소는 그 variable에 대해 계산된 미분값이 들어가면 됩니다.
앞서 `tape.gradient`를 호출할 때, 두 번째 인자로 `trainable_variables`을 넘겨줬었습니다. 이 `trainable_variables`는 해당 모델의 학습 가능한 파라미터들의 리스트로, `tape.gradient`를 통해 gradient를 계산한 gradient는 `trainable_variables`의 각 파라미터 순서와 정확히 일치한 순서로 계산된다고 했습니다. 따라서, `optimizer.apply_gradients`로 넘길 파라미터도 `tape.gradient`에 넘겼던 파라미터 순서대로 넘기면 됩니다. 다만, 파라미터 리스트 그대로 들어가는 것이 아니라, 각각 서로다른 튜플의 원소로 들어가있죠..

이를 해결하기 위한 좋은 함수로 `zip`함수가 있습니다. python의 내장함수로, 같은 길이의 두 리스트를 받아서 두 리스트의 첫 번째 인자를 각각 추출하고 묶어줍니다. 그리고 두 번째 인자를 각각 추출하고 묶어줍니다. 저희는 이런 `zip`함수를 이용하여 gradient와 parameter를 묶어줄 수 있습니다.

이제 테스트 함수를 구현해보겠습니다. 이것은 validation step이라고 생각하시면 되는데요, `tf.GradientTape`를 생성하지 않고, 학습 과정이 없는 것을 빼면 `train_step`과 일치합니다.

In [6]:
def test_step(x, y):
    """
    테스트 함수
    
    Arguments:
    ----------
    - x: 이미지 배치
    - y: 각 이미지에 해당하는 라벨
    
    Returns:
    --------
    - loss: test loss
    - acc: test accuracy
    """
    
    # loss와 accuracy 초기화
    loss = 0.0
    acc = 0.0
    
    # forward propagation, 단 gradient는 계산하지 않음
    preds = myfirstmodel(x)
    
    # loss, accuracy 계산
    loss += criterion(y, preds)
    acc = accuracy(y, preds)
    
    return loss.numpy(), acc.numpy()

이제, loss 객체를 생성하고, accuracy 계산을 도와줄 객체와 optimizer 객체를 생성합니다.

In [7]:
criterion = losses.SparseCategoricalCrossentropy() # loss 객체
accuracy = metrics.SparseCategoricalAccuracy()     # accuracy 객체
optimizer = optimizers.Adam(learning_rate=1e-3)    # optimizer 객체

다음은 이제 학습을 돌릴 차례입니다.

In [8]:
EPOCHS = 100

for e in range(EPOCHS):
    train_loss = train_step(X_train, y_train)
    test_loss, test_acc = test_step(X_test, y_test)
    
    if e == 0 or (e+1)%10 == 0:
        print(f"Epochs {e+1}/{EPOCHS}, train loss: {train_loss:.6f}, test loss: {test_loss:.6f}, test acc: {test_acc:.4f}")

Epochs 1/100, train loss: 2.420222, test loss: 2.221694, test acc: 0.1970
Epochs 10/100, train loss: 1.152546, test loss: 1.057999, test acc: 0.5805
Epochs 20/100, train loss: 0.670235, test loss: 0.626358, test acc: 0.6963
Epochs 30/100, train loss: 0.497295, test loss: 0.471012, test acc: 0.7506
Epochs 40/100, train loss: 0.412965, test loss: 0.393284, test acc: 0.7836
Epochs 50/100, train loss: 0.365790, test loss: 0.348790, test acc: 0.8063
Epochs 60/100, train loss: 0.335861, test loss: 0.321412, test acc: 0.8227
Epochs 70/100, train loss: 0.314138, test loss: 0.302092, test acc: 0.8352
Epochs 80/100, train loss: 0.296966, test loss: 0.287123, test acc: 0.8453
Epochs 90/100, train loss: 0.282537, test loss: 0.274664, test acc: 0.8536
Epochs 100/100, train loss: 0.269824, test loss: 0.263786, test acc: 0.8606
